Arthur Catarino de Oliveira  
Guilherme Fidélis Freire  
Mateus Poddis Correa

Neste exercicio foi criado um subgrupo A para as semanas 1 e 2 que permitem a fabricação de camisas além do limite permitido semanalmente.

##Dados
- S: Semanas do problema.
- A: Semanas que permitem produção adicional.
- Di:Demanda da semana i, i E S.
- Ci: Custo de produção da semana i, i E S.
- CEi: Custo de estoque da semana i, i E S.
- CAj: Custo de produção adicional da semana j, j E A.
- MaxProducao: Quantidade máxima de camisas produzidas por semana.
- MaxAdicional: Quantidade máxima de camisas adicionais produzidas por semana.

##Variaveis de decisão
- Xi, i E S, quantidade de camisas produzidas na semana.
- Ei, i E S, quantidade de camisas estocadas na semana.
- XAj, j E A, quantidade de camisas adicionais produzidas na semana.

##Função Objetivo:
$$
Minimizar: \sum_{i E S}^{} (X_i*C_i+E_i*CE_i)  + \sum_{j E A}^{} XA_j*CA_j
$$

##Restrições:
1) Restrição que respeita o máximo de camisas produzidas na semana.

$X_i <= MaxProducao$  $∀ i E S$

2) Restrição que respeita o máximo de camisa adicionais produzida nas semanas permitidas.

$XA_j <= MaxAdicional$  $∀ j E A$

3) Restrição que garante que a produção da semana e o estoque da semana anterior atinjam a demanda.
Caso: Semana 1(Não tem estoque e permite produção adicional).

$X_1 + XA_1>= D_1$

Caso: Semana 2(Com estoque e produção adicional).

$X_2 + XA_2+E_1>= D_2$

Restante das semanas(Com estoque, sem produção adicional).

$X_i +E(i-1)>= D_i$

4) Restrição que define o estoque. Para a semana 1 (Com produção adicional e sem estoque anterior).

$X_1+XA_1 - D_i= E_1$

Para a semana 2 (Com produção adicional e com estoque).

$X_2+XA_2+E_1- D_i= E_2$

Para o restante das semanas (Com estoque apenas).

$X_i+E(i-1)-D_i= E_i$ $∀ i E S$

5) Restrição para garantir que as variáveis não sejam menores que zero.

$X_i>= 0$ $∀ i E S$

$E_i>= 0$ $∀ i E S$

$XA_j>= 0$ $∀ j E A$

In [ ]:
!pip install pulp
import pulp

Os dados foram organizados em um dicionário(semana,Objeto Semana). Sendo que o objeto Semana possui informações de produção e estoque.

In [ ]:
class Semana:
  def __init__(self,demanda, custoProducao, custoEstoque, custoAdicional, prodAdicional):
    self.demanda = demanda
    self.custoProducao = custoProducao
    self.custoEstoque = custoEstoque
    self.custoAdicional = custoAdicional
    self.prodAdicional = prodAdicional

semanas = {
  1 :Semana(5000, 2, 0.2, 2.8, True),
  2 :Semana(10000, 2, 0.2, 2.8, True),
  3 :Semana(30000, 2.5, 0.2, 0, False),
  4 :Semana(60000, 2.5, 0.2, 0, False)
}

MaxProducao = 25000
MaxAdicional = 10000

Inicializando o modelo

In [ ]:
modelo = pulp.LpProblem('Fábrica de Camisetas', pulp.LpMinimize)

quantidadeSem = pulp.LpVariable.dicts('semana', list(semanas.keys()), lowBound=0)
quantidadeEst = pulp.LpVariable.dicts('estoque', list(semanas.keys()), lowBound=0)
quantidadeAd = pulp.LpVariable.dicts('adicional', list(semanas.keys()), lowBound=0)

/usr/local/lib/python3.12/dist-packages/pulp/pulp.py:1489: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


##Função Objetivo:
$$
Minimizar: \sum_{i E S}^{} (X_i*C_i+E_i*CE_i)  + \sum_{j E A}^{} XA_j*CA_j
$$

In [ ]:
modelo += pulp.lpSum(quantidadeSem[i]*semanas[i].custoProducao + quantidadeEst[i]*semanas[i].custoEstoque
                     for i in quantidadeSem.keys()) + pulp.lpSum(quantidadeAd[j]*semanas[j].custoAdicional for j in quantidadeAd.keys())

Restrições:

1) $X_i <= MaxProducao$  $∀ i E S$ (Limita a produção)

In [ ]:
for i in quantidadeSem.keys():
  modelo += quantidadeSem[i] <= MaxProducao

2) $XA_j <= MaxAdicional$  $∀ j E A$ (Limita a produção adicional)

In [ ]:
for j in quantidadeAd.keys():
  modelo += quantidadeAd[j] <= MaxAdicional

3) Garante que a demanda seja atendida.

a) $X_1 + XA_1>= D_1$ (Semana 1, sem estoque e produção adicional).

b) $X_2 + XA_2+E_1>= D_2$ (Semana 2, com estoque e produção adicional).

c) $X_i +E(i-1)>= D_i$ (Restante das semanas, apenas com estoque).

In [ ]:
for i in quantidadeSem.keys():
  if(semanas[i].prodAdicional):
    if(i == 1):
      modelo += quantidadeSem[i] + quantidadeAd[i] >= semanas[i].demanda
    else:
      modelo += quantidadeSem[i] + quantidadeAd[i] + quantidadeEst[i-1] >= semanas[i].demanda
  else:
    modelo += quantidadeSem[i] + quantidadeEst[i - 1] >= semanas[i].demanda

4) Define o estoque da semana.

a) $X_1+XA_1 - D_i= E_1$ (Sem estoque anterior, e com produção adicional).

b) $X_2+XA_2+E_1- D_i= E_2$ (Com estoque anterior e produção adicional).

c) $X_i+E(i-1)-D_i= E_i$ $∀ i E S$(Com estoque apenas).

In [ ]:
for i in quantidadeSem.keys():
  if(semanas[i].prodAdicional):
    if(i == 1):
      modelo += quantidadeSem[i] + quantidadeAd[i] - semanas[i].demanda == quantidadeEst[i]
    else:
      modelo += quantidadeSem[i] + quantidadeAd[i] + quantidadeEst[i - 1] - semanas[i].demanda == quantidadeEst[i]
  else:
    modelo += quantidadeSem[i] + quantidadeEst[i - 1] - semanas[i].demanda == quantidadeEst[i]

5) Garante que nenhuma variável seja negativa.

$X_i>= 0$ $∀ i E S$

$E_i>= 0$ $∀ i E S$

$XA_j>= 0$ $∀ j E A$

In [ ]:
for i in quantidadeSem.keys():
  modelo += quantidadeSem[i] >= 0
  modelo += quantidadeEst[i] >= 0
  if(semanas[i].prodAdicional):
    modelo += quantidadeAd[i] >= 0

Imprime o modelo.

In [ ]:
print(modelo)

Fábrica_de_Camisetas:
MINIMIZE
2.8*adicional_1 + 2.8*adicional_2 + 0.2*estoque_1 + 0.2*estoque_2 + 0.2*estoque_3 + 0.2*estoque_4 + 2*semana_1 + 2*semana_2 + 2.5*semana_3 + 2.5*semana_4 + 0.0
SUBJECT TO
_C1: semana_1 <= 25000

_C2: semana_2 <= 25000

_C3: semana_3 <= 25000

_C4: semana_4 <= 25000

_C5: adicional_1 <= 10000

_C6: adicional_2 <= 10000

_C7: adicional_3 <= 10000

_C8: adicional_4 <= 10000

_C9: adicional_1 + semana_1 >= 5000

_C10: adicional_2 + estoque_1 + semana_2 >= 10000

_C11: estoque_2 + semana_3 >= 30000

_C12: estoque_3 + semana_4 >= 60000

_C13: adicional_1 - estoque_1 + semana_1 = 5000

_C14: adicional_2 + estoque_1 - estoque_2 + semana_2 = 10000

_C15: estoque_2 - estoque_3 + semana_3 = 30000

_C16: estoque_3 - estoque_4 + semana_4 = 60000

_C17: semana_1 >= 0

_C18: estoque_1 >= 0

_C19: adicional_1 >= 0

_C20: semana_2 >= 0

_C21: estoque_2 >= 0

_C22: adicional_2 >= 0

_C23: semana_3 >= 0

_C24: estoque_3 >= 0

_C25: semana_4 >= 0

_C26: estoque_4 >= 0

VARIA

Solução do problema.

In [ ]:
status = modelo.solve()

print('Status: ', pulp.LpStatus[status])
print('Função objetivo: ', pulp.value(modelo.objective))
print('Soluções')
for semana in semanas.keys():
    print('Semana ', semana,' Quantidade produzida: ', quantidadeSem[semana].value())
    if(semanas[semana].prodAdicional):
      print('Quantidade adicional na semana ', semana, ': ', quantidadeAd[semana].value())
    print(' Estoque mantido: ', quantidadeEst[semana].value(), '\n')

Status:  Optimal
Função objetivo:  258000.0
Soluções
Semana  1  Quantidade produzida:  25000.0
Quantidade adicional na semana  1 :  0.0
 Estoque mantido:  20000.0 

Semana  2  Quantidade produzida:  25000.0
Quantidade adicional na semana  2 :  5000.0
 Estoque mantido:  40000.0 

Semana  3  Quantidade produzida:  25000.0
 Estoque mantido:  35000.0 

Semana  4  Quantidade produzida:  25000.0
 Estoque mantido:  0.0 

